In [1]:
import pandas as pd
import numpy as np
import gif
import time
import matplotlib.pyplot as plt

import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [14]:
df = pd.read_csv('reddit_data.csv')

campaign_1_heavy = df[df['root_post'] == 'post_1']
campaign_2_heavy = df[df['root_post'] == 'post_2']

cols = list(campaign_2_heavy.columns)
new_cols = cols[:7] + ['Warmth Scale'] + cols[8:]
cross_map = {cols[i]:new_cols[i] for i in range(len(cols))}

campaign_1_heavy = campaign_1_heavy.rename(columns=cross_map)
campaign_2_heavy = campaign_2_heavy.rename(columns=cross_map)

In [15]:
campaign_1_heavy

,post_time_utc,post_body,response_body,response_time,root_post,post_time,response_time_utc,Warmth Scale,lat,lon,name
0,1635433244,Hell's canyon?,Warm!,2021-10-28 16:04:51,post_1,2021-10-28 08:00:44,1635462291,1.0,34.9180754,-112.2785013,"hell canyon, arizona, usa"
1,1635433325,[deleted],Getting colder!,2021-10-28 16:04:40,post_1,2021-10-28 08:02:05,1635462280,-1.0,to_delete,to_delete,to_fix
2,1635443314,Maine,Cold!,2021-10-28 16:03:17,post_1,2021-10-28 10:48:34,1635462197,-1.0,45.253783,-69.4454689,"maine, usa"
3,1635452573,"Sterling, CO",This guess is a little bit chilly,2021-10-28 16:02:08,post_1,2021-10-28 13:22:53,1635462128,-1.0,40.6255415,-103.207709,"sterling, co 80751, usa"
4,1635450547,"Joshua Tree, California",Cold!,2021-10-28 16:01:29,post_1,2021-10-28 12:49:07,1635462089,-1.0,34.134728,-116.3130661,"joshua tree, ca 92252, usa"
...,...,...,...,...,...,...,...,...,...,...,...
184,1635231007,"Juneau, Alaska","Just like the location, this answer is pretty ...",2021-10-26 09:34:46,post_1,2021-10-25 23:50:07,1635266086,-1.0,58.3019444,-134.4197221,"juneau, ak, usa"
185,1635207396,"Wilmington, Delaware","Good guess, but Wilmington, Delware is cold!",2021-10-26 09:33:55,post_1,2021-10-25 17:16:36,1635266035,-1.0,39.744655,-75.5483909,"wilmington, de, usa"
186,1635189854,"Burlington, Vermont",This guess is very cold. Try again (in 24 hrs)!,2021-10-26 09:32:34,post_1,2021-10-25 12:24:14,1635265954,-2.0,44.4758825,-73.21207199999999,"burlington, vt, usa"
187,1635190287,Moab Utah,Getting colder. Tell us another guess in 24 hrs!,2021-10-26 09:30:24,post_1,2021-10-25 12:31:27,1635265824,-1.0,38.5733155,-109.5498395,"moab, ut 84532, usa"


In [17]:
frames = []

@gif.frame
def gen_fig(i, df):

    lower_bound = min(list(df['Warmth Scale'].unique()))
    upper_bound = max(list(df['Warmth Scale'].unique()))
    

    fig = px.scatter_geo(df[:i],
                        lat='lat',
                        lon='lon',
                        color='Warmth Scale',
                        scope='usa',
                        range_color=[lower_bound, upper_bound],
                        title=df['name'][i]
                        )

    return fig
    

        


maps = []
for i in range(1, len(campaign_1_heavy)):
    frame = gen_fig(i, campaign_1_heavy)
    frames.append(frame)


gif.save(frames, 'campaign_1.gif', duration=500)

In [ ]:
# create a list of frames
df = campaign_2_heavy
lower_bound = min(list(df['Warmth Scale'].unique()))
upper_bound = max(list(df['Warmth Scale'].unique()))
frames = []


def gen_frame(i, df):
    # create the button
    button = {
        "type": "buttons",
        "buttons": [
            {
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 500}}],
            }
        ],
    }
    # add the button to the layout and update the 
    # title to show the gradient descent step
    layout = go.Layout(updatemenus=[button], 
                       title_text=f"Guesses up to {i}",

                      )

    datapoint = go.Scattergeo(lat = df['lat'][:i],
                        lon = df['lons'][:i],
                        mode = 'markers',
                        marker_color = df['Warmth Scale'][:i],
                        marker = {'autocolorscale':False, 'cmax':2, 'cmin':-3, 'color':'#E5912D'}
                            )
    frame = go.Frame(
        data=datapoint, 
        layout=go.Layout(title_text=f"Guesses up to number {i}")
    )

    return frame

    
frames = []
for i in range(len(df)):
    frames.append(gen_frame(i, campaign_1_light))
    
    
    
    
# combine the graph_objects into a figure
fig = go.Figure(data=datapoint,
                frames=frames,
                layout=layout,
                
                )
                                 
# show our animation!
fig.update_layout(geo_scope='usa',
                  paper_bgcolor='#CCE1E9',
                  plot_bgcolor='black'

)
fig.show()

#fig.write_image('image.svg')